In [1]:
import os
import sys
import aim
import glob
import random

os.environ["CONFIG_PATHS"] = "../configs/training_unlooped.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../data/moves_10"'
sys.path.append("../src")

import numpy as np
import torch 
from torch import nn
from tqdm import tqdm
from torch.amp import GradScaler, autocast
import glob
import time
from configuration import moves_data, config
from training.load_games import load_games_new

In [2]:
from neural_net import NeuralNet
from training.game_data_manager import GameDataManager, DirectoryGameDataPathFetcher, CustomGameDataPathFetcher

Loaded config:  {"game": {"board_size": 10, "num_moves": 6233, "num_pieces": 21, "moves_directory": "../data/moves_10"}, "training": {"batch_size": 128, "policy_loss_weight": 0.158, "learning_rate": 0.001}, "networks": {"default": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 64, "residual_blocks": 8}}, "agents": []}


In [3]:
NETWORK_CONFIG = config()["networks"]["default"]

In [4]:
BATCH_SIZE = 128
DEVICE = "cpu"
MOVES = moves_data()
GAMES_DIR = "../data/2024-11-23_00-37-50-doublehandedness/games"
POLICY_LOSS_WEIGHT = 0.158
LEARNING_RATE = 1e-3

Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners


In [5]:
model = NeuralNet(NETWORK_CONFIG)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [6]:
def get_test_losses(model, test_dataset):
    dataloader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )
    model.eval()

    results = {
        "total_loss": 0,
        "value_loss": 0,
        "policy_loss": 0,
        "value_max_correct": 0,
        "policy_max_correct": 0,
    }

    total_sample_count = 0

    with torch.inference_mode():
        for boards, policies, values in dataloader:
            boards = boards.to(dtype=torch.float32, device=DEVICE)
            policies = policies.to(dtype=torch.float32, device=DEVICE)
            values = values.to(dtype=torch.float32, device=DEVICE)

            pred_values, pred_policy_logits = model(boards)

            value_loss = nn.CrossEntropyLoss(reduction="sum")(
                pred_values,
                values,
            )
            policy_loss = nn.CrossEntropyLoss(reduction="sum")(
                pred_policy_logits,
                policies,
            )
            loss = value_loss + POLICY_LOSS_WEIGHT * policy_loss

            results["total_loss"] += loss.item()
            results["value_loss"] += value_loss.item()
            results["policy_loss"] += policy_loss.item()
            results["value_max_correct"] += (pred_values.argmax(dim=1) == values.argmax(dim=1)).sum().item()
            results["policy_max_correct"] += (pred_policy_logits.argmax(dim=1) == policies.argmax(dim=1)).sum().item()

            total_sample_count += len(boards)

    results["total_loss"] /= total_sample_count
    results["value_loss"] /= total_sample_count
    results["policy_loss"] /= total_sample_count
    results["value_max_correct"] /= total_sample_count
    results["policy_max_correct"] /= total_sample_count

    return results

In [7]:
TRAIN_TEST_SPLIT = 0.9

def get_dataset(game_files):
    gamedata = load_games_new(game_files, with_tqdm=True)
    boards_tensor = torch.from_numpy(gamedata["boards"]).to(dtype=torch.float)
    policies_tensor = torch.from_numpy(gamedata["policies"]).to(dtype=torch.float)
    values_tensor = torch.from_numpy(gamedata["values"]).to(dtype=torch.float)
    return torch.utils.data.TensorDataset(boards_tensor, policies_tensor, values_tensor)

file_paths = glob.glob("/Users/shivamsarodia/Dev/blockus/data/2024-11-23_00-37-50-doublehandedness/untrained_games_*/*.npz")

random.seed(20554)
random.shuffle(file_paths)

# Truncate the to 200 for testing
file_paths = file_paths[:200]

num_train_games = int(len(file_paths) * TRAIN_TEST_SPLIT)
train_file_paths = file_paths[:num_train_games]
test_file_paths = file_paths[num_train_games:]

train_dataset = get_dataset(train_file_paths)
test_dataset = get_dataset(test_file_paths)

print("Num train samples:", len(train_dataset))
print("Num test samples:", len(test_dataset))

100%|██████████| 20/20 [00:00<00:00, 55.14it/s]


Num train samples: 158528
Num test samples: 17737


In [8]:
def get_test_losses(model):
    dataloader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )
    model.eval()

    results = {
        "total_loss": 0,
        "value_loss": 0,
        "policy_loss": 0,
        "value_max_correct": 0,
        "policy_max_correct": 0,
    }

    total_sample_count = 0

    with torch.inference_mode():
        for boards, policies, values in dataloader:
            boards = boards.to(dtype=torch.float32, device=DEVICE)
            policies = policies.to(dtype=torch.float32, device=DEVICE)
            values = values.to(dtype=torch.float32, device=DEVICE)

            pred_values, pred_policy_logits = model(boards.to(dtype=torch.float32, device=DEVICE))
            value_loss = nn.CrossEntropyLoss(reduction="sum")(
                pred_values,
                values,
            )
            policy_loss = nn.CrossEntropyLoss(reduction="sum")(
                pred_policy_logits,
                policies,
            )
            loss = value_loss + POLICY_LOSS_WEIGHT * policy_loss

            results["total_loss"] += loss.item()
            results["value_loss"] += value_loss.item()
            results["policy_loss"] += policy_loss.item()
            results["value_max_correct"] += (pred_values.argmax(dim=1) == values.argmax(dim=1)).sum().item()
            results["policy_max_correct"] += (pred_policy_logits.argmax(dim=1) == policies.argmax(dim=1)).sum().item()

            total_sample_count += len(boards)

    results["total_loss"] /= total_sample_count
    results["value_loss"] /= total_sample_count
    results["policy_loss"] /= total_sample_count
    results["value_max_correct"] /= total_sample_count
    results["policy_max_correct"] /= total_sample_count

    return results

In [9]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

run = aim.Run(repo='/Users/shivamsarodia/Dev/blockus/')
run["hparams"] = {
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "policy_loss_weight": POLICY_LOSS_WEIGHT,
}

scaler = GradScaler(device=DEVICE)

model.train()

batch_index = 0
for epoch in range(1):
    for boards, policies, values in tqdm(train_dataloader):
        boards = boards.to(dtype=torch.float32, device=DEVICE)
        policies = policies.to(dtype=torch.float32, device=DEVICE)
        values = values.to(dtype=torch.float32, device=DEVICE)

        with autocast(device_type=DEVICE, dtype=torch.float16):
            pred_values, pred_policy = model(boards)
            value_loss = nn.CrossEntropyLoss()(
                pred_values,
                values,
            )

            policy_loss = nn.CrossEntropyLoss()(
                pred_policy,
                policies,
            )
            loss = value_loss + POLICY_LOSS_WEIGHT * policy_loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # loss.backward()
        # optimizer.step()
        
        optimizer.zero_grad()

        training_result = {
            "total_loss": loss.item(),
            "value_loss": value_loss.item(),
            "policy_loss": policy_loss.item(),
            "value_max_correct": (pred_values.argmax(dim=1) == values.argmax(dim=1)).sum().item() / len(boards),
            "policy_max_correct": (pred_policy.argmax(dim=1) == policies.argmax(dim=1)).sum().item() / len(boards),
        }

        for key, value in training_result.items():
            run.track(
                value,
                name=key,
                step=batch_index,
                context={"subset": "train"},
            )

        if batch_index % 1000 == 0:
            test_losses = get_test_losses(model)
            for key, value in test_losses.items():
                run.track(
                    value,
                    name=key,
                    step=batch_index,
                    context={"subset": "test"},
                )
                print(test_losses)
        batch_index += 1

    print("Finished epoch")

run.close()

  0%|          | 0/1239 [00:00<?, ?it/s]


NotImplementedError: The operator 'aten::_amp_foreach_non_finite_check_and_unscale_' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [21]:
run.close()

In [9]:
# torch.save(model.state_dict(), "../data/2024-12-02_21-22-57-notebook-models/sample_ratio_two_2.pt")

In [ ]:
count_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
)

boards_seen = set()
total_boards_count = 0
boards_with_at_least_one_piece_count = 0
boards_with_at_least_two_pieces_count = 0
boards_with_at_least_three_pieces_count = 0
boards_with_at_least_four_pieces_count = 0
for boards, policies, values, valid_moves in tqdm(count_dataloader):
    boards_seen.add(boards.numpy(force=True).tobytes())
    total_boards_count += 1
    if torch.sum(boards) > 0:
        boards_with_at_least_one_piece_count += 1
    if torch.sum(boards) > 5:
        boards_with_at_least_two_pieces_count += 1
    if torch.sum(boards) > 10:
        boards_with_at_least_three_pieces_count += 1
    if torch.sum(boards) > 15:
        boards_with_at_least_four_pieces_count += 1
        
print("Number of unique boards:", len(boards_seen))
print("Number of boards with at least one piece:", boards_with_at_least_one_piece_count)
print("Number of boards with at least two pieces:", boards_with_at_least_two_pieces_count)
print("Number of boards with at least three pieces:", boards_with_at_least_three_pieces_count)
print("Number of boards with at least four pieces:", boards_with_at_least_four_pieces_count)


In [23]:
# Load models
def load_model(path):
    model = NeuralNet(NETWORK_CONFIG)
    model.load_state_dict(torch.load(path, weights_only=True))
    model.to(DEVICE)
    return model

# base_model_1 = load_model("/Users/shivamsarodia/Dev/blockus/data/2024-12-02_21-22-57-notebook-models/base_in_notebook_1.pt")
# base_model_2 = load_model("/Users/shivamsarodia/Dev/blockus/data/2024-12-02_21-22-57-notebook-models/base_in_notebook_2.pt")
# ratio_two_model_1 = load_model("/Users/shivamsarodia/Dev/blockus/data/2024-12-02_21-22-57-notebook-models/sample_ratio_two_1.pt")
ratio_two_model_2 = load_model("/Users/shivamsarodia/Dev/blockus/data/2024-12-02_21-22-57-notebook-models/sample_ratio_two_2.pt")

# print("done")

In [ ]:
get_test_losses(ratio_two_model_2)

In [ ]:
test_game_files = glob.glob("/Users/shivamsarodia/Dev/blockus/data/2024-11-23_00-37-50-doublehandedness/more_games_we_didnt_train_on/*.npz")
test_gamedata = load_games_new(test_game_files, with_tqdm=True)

print("Converting to tensors...")

# Convert numpy arrays to torch tensors
boards_tensor = torch.from_numpy(test_gamedata["boards"]).to(dtype=torch.float, device="mps")
policies_tensor = torch.from_numpy(test_gamedata["policies"]).to(dtype=torch.float, device="mps")
values_tensor = torch.from_numpy(test_gamedata["values"]).to(dtype=torch.float, device="mps")
valid_moves = torch.from_numpy(test_gamedata["valid_moves"]).to(dtype=torch.bool, device="mps")

dataset = torch.utils.data.TensorDataset(boards_tensor, policies_tensor, values_tensor, valid_moves)

In [ ]:
# base_model_1_test_result = get_test_losses(base_model_1)
base_model_2_test_result = get_test_losses(base_model_2)
# ratio_two_model_1_test_result = get_test_losses(ratio_two_model_1)
ratio_two_model_2_test_result = get_test_losses(ratio_two_model_2)

In [ ]:
len(test_gamedata["boards"])

In [ ]:
241365 + 336673 + 500000

In [ ]:
base_model_2_test_result

In [ ]:
ratio_two_model_2_test_result

In [ ]:
def range_of_boolean_var(p):
    standard_error = np.sqrt(p * (1 - p) / len(dataset))
    return p + 2 * standard_error, p - 2 * standard_error

import matplotlib.pyplot as plt

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

models = ['Base 1', 'Base 2', 'Ratio Two 1', 'Ratio Two 2']

# Value accuracy plot
value_accuracies = [
    base_model_1_test_result["value_max_correct"],
    base_model_2_test_result["value_max_correct"], 
    ratio_two_model_1_test_result["value_max_correct"],
    ratio_two_model_2_test_result["value_max_correct"]
]
value_errors = [range_of_boolean_var(acc) for acc in value_accuracies]
value_upper = [err[0] - acc for acc, err in zip(value_accuracies, value_errors)]
value_lower = [acc - err[1] for acc, err in zip(value_accuracies, value_errors)]

ax1.bar(models, value_accuracies, yerr=[value_lower, value_upper], capsize=5)
ax1.set_title('Value Prediction Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(bottom=min(value_accuracies) * 0.95)  # Set bottom to 95% of minimum value

# Policy accuracy plot  
policy_accuracies = [
    base_model_1_test_result["policy_max_correct"],
    base_model_2_test_result["policy_max_correct"],
    ratio_two_model_1_test_result["policy_max_correct"], 
    ratio_two_model_2_test_result["policy_max_correct"]
]
policy_errors = [range_of_boolean_var(acc) for acc in policy_accuracies]
policy_upper = [err[0] - acc for acc, err in zip(policy_accuracies, policy_errors)]
policy_lower = [acc - err[1] for acc, err in zip(policy_accuracies, policy_errors)]

ax2.bar(models, policy_accuracies, yerr=[policy_lower, policy_upper], capsize=5)
ax2.set_title('Policy Prediction Accuracy')
ax2.set_ylabel('Accuracy')
ax2.set_ylim(bottom=min(policy_accuracies) * 0.95)  # Set bottom to 95% of minimum value

plt.tight_layout()
plt.show()


In [ ]:
print("Base model 1")
print(base_model_1_test_result)
print("Base model 2")
print(base_model_2_test_result)
print("Ratio two model 1")
print(ratio_two_model_1_test_result)
print("Ratio two model 2")
print(ratio_two_model_2_test_result)

In [ ]:
import glob
import random

file_paths = glob.glob("/Users/shivamsarodia/Dev/blockus/data/2024-11-23_00-37-50-doublehandedness/untrained_games_*/*.npz")

random.seed(20554)
random.shuffle(file_paths)

num_train_games = int(len(file_paths) * 0.9)
train_file_paths = file_paths[:num_train_games]
test_file_paths = file_paths[num_train_games:]

print("Num train files:", len(train_file_paths))
print("Num test files:", len(test_file_paths))

print("Loading train games...")
train_dataset = get_dataset(train_file_paths)

print("Loading test games...")
test_dataset = get_dataset(test_file_paths)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
)

In [ ]:
from collections import Counter

test_boards = Counter()
for board, _, _, _ in test_dataloader:
    test_boards[board.numpy(force=True).tobytes()] += 1

print("Number of total test boards:", len(test_dataset))
print("Number of unique test boards:", len(test_boards))

train_boards = Counter()
for board, _, _, _ in train_dataloader:
    train_boards[board.numpy(force=True).tobytes()] += 1

print("Number of total train boards:", len(train_dataset))
print("Number of unique train boards:", len(train_boards))

test_boards_in_train = Counter()
for board in test_boards:
    if board in train_boards:
        test_boards_in_train[board] = test_boards[board]

print("Number of boards shared:", len(test_boards_in_train))
print("Number of rows of test data that appear in train data:", sum(test_boards_in_train.values()))

In [ ]:
99330 / 181003

In [ ]:
99330 / 181003

In [ ]:
print("Number of total test boards:", len(test_dataset))
print("Number of total train boards:", len(train_dataset))
print("Number of boards shared:", len(test_boards & train_boards))

In [ ]:
from collections import Counter

c1 = Counter()
c1[4] += 1
c2 = Counter()
c2[5] += 3
c1 & c2

In [ ]:
sum(c2.values())

In [ ]:
101767 / 181003

In [ ]:
for board, _, _, _ in train_dataset:
    print(board.shape)
    break